Load model and embedding data.

In [1]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from attention_model import AttentionModel

theano.config.floatX='float32'

model_name = 'model/attention_emb125_h125_model.npz'

state = prototype_state()
model = AttentionModel(state, test_mode=True)
model.load(model_name)

(word2ind, ind2word) = cPickle.load(open('tmp/dict.pkl'))

print('Data loaded.')

Data loaded.


Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [8]:
test_sents = ['chinese food']

for (k, test_sent) in enumerate(test_sents):
    words = test_sent.split()
    nat_coded = [1]
    for w in words:
        if w in word2ind:
            nat_coded.append(word2ind[w])
        else:
            print '  out of vocab: %s' % w
    nat_coded.append(0)
    print 'Coded input:', nat_coded

    m = state['seq_len_in']
    nat_coded_mat = numpy.zeros((m, 2), dtype='int32')
    nat_mask = numpy.zeros((m, 2), dtype='float32')
    sent_len = len(nat_coded)
    nat_coded_mat[:sent_len, 0] = nat_coded
    nat_mask[:sent_len, 0] = 1
    pred_fn = theano.function([model.x_data, model.xmask],
                                    model.ot)
    pred = pred_fn(nat_coded_mat, nat_mask)
    #print 'Prediction (coded):', pred

    res_col = pred[:, 0]
    print res_col

    res = []
    for ind in res_col:
        res.append(ind)
        if ind == 0:
            break

    print "%d: %s" % (k, ' '.join([ind2word[ind] for ind in res]))

Coded input: [1, 7, 467, 0]
[250  13   1   1   1   1   1   1   1   1]
0: american system <START> <START> <START> <START> <START> <START> <START> <START>
